# prelearning for NLP
* TF: term frequency, 某个词在某篇文章中出现的频率,用于衡量一篇文章中某个词的重要性</br>
TF = 某篇文章中某词条出现的次数/该篇文章中所有词条数目
* IDF: inverse document frequency, 全局中文章包含某个词的频率，用于衡量某一词在某一篇文章中的独特区分性（去除"你，我，他"之中在很多文章中都出现词的干扰）
IDF = log(语料库中的文档总数/(包含某词条的文档数)+1)
* TF-IDF：两者结合使用将该篇文章的关键字提取出来

# 计算相关性
* 使用TF与IDF计算出该文档中每个词的TF，IDF值
* 将词对应的TF与IDF值相乘
* 将这些值视作一个vector，如下图:</br>
![avatar](pic/doc-vectors.png)
* 搜索时，将关键词vector化，计算与各个文档vector的cos距离，进行相关性排序

In [4]:
import numpy as np
from collections import Counter
import itertools
# from visual import show_tfidf

docs = [
    "it is a good day, I like to stay here",
    "I am happy to be here",
    "I am bob",
    "it is sunny today",
    "I have a party today",
    "it is a dog and that is a cat",
    "there are dog and cat on the tree",
    "I study hard this morning",
    "today is a good day",
    "tomorrow will be a good day",
    "I like coffee, I like book and I like apple",
    "I do not like it",
    "I am kitty, I like bob",
    "I do not care who like bob, but I like kitty",
    "It is coffee time, bring your cup",
]

docs_words = [d.replace(",", "").split(" ") for d in docs]
vocab = set(itertools.chain(*docs_words))
v2i = {v: i for i, v in enumerate(vocab)}
i2v = {i: v for v, i in v2i.items()}
print(docs_words)
print(v2i)
print(i2v)

[['it', 'is', 'a', 'good', 'day', 'I', 'like', 'to', 'stay', 'here'], ['I', 'am', 'happy', 'to', 'be', 'here'], ['I', 'am', 'bob'], ['it', 'is', 'sunny', 'today'], ['I', 'have', 'a', 'party', 'today'], ['it', 'is', 'a', 'dog', 'and', 'that', 'is', 'a', 'cat'], ['there', 'are', 'dog', 'and', 'cat', 'on', 'the', 'tree'], ['I', 'study', 'hard', 'this', 'morning'], ['today', 'is', 'a', 'good', 'day'], ['tomorrow', 'will', 'be', 'a', 'good', 'day'], ['I', 'like', 'coffee', 'I', 'like', 'book', 'and', 'I', 'like', 'apple'], ['I', 'do', 'not', 'like', 'it'], ['I', 'am', 'kitty', 'I', 'like', 'bob'], ['I', 'do', 'not', 'care', 'who', 'like', 'bob', 'but', 'I', 'like', 'kitty'], ['It', 'is', 'coffee', 'time', 'bring', 'your', 'cup']]
{'am': 0, 'bob': 1, 'day': 2, 'happy': 3, 'tomorrow': 4, 'like': 5, 'a': 6, 'I': 7, 'coffee': 8, 'cat': 9, 'care': 10, 'but': 11, 'have': 12, 'this': 13, 'your': 14, 'today': 15, 'It': 16, 'will': 17, 'hard': 18, 'book': 19, 'there': 20, 'time': 21, 'bring': 22, 'a

In [2]:
def safe_log(x):
    mask = x != 0
    x[mask] = np.log(x[mask])
    return x


tf_methods = {
        "log": lambda x: np.log(1+x),
        "augmented": lambda x: 0.5 + 0.5 * x / np.max(x, axis=1, keepdims=True),
        "boolean": lambda x: np.minimum(x, 1),
        "log_avg": lambda x: (1 + safe_log(x)) / (1 + safe_log(np.mean(x, axis=1, keepdims=True))),
    }
idf_methods = {
        "log": lambda x: 1 + np.log(len(docs) / (x+1)),
        "prob": lambda x: np.maximum(0, np.log((len(docs) - x) / (x+1))),
        "len_norm": lambda x: x / (np.sum(np.square(x))+1),
    }

def get_tf(method="log"):
    # term frequency: how frequent a word appears in a doc
    _tf = np.zeros((len(vocab), len(docs)), dtype=np.float64)    # [n_vocab, n_doc]
    for i, d in enumerate(docs_words):
        counter = Counter(d)
        for v in counter.keys():
            _tf[v2i[v], i] = counter[v] / counter.most_common(1)[0][1]

    weighted_tf = tf_methods.get(method, None)
    if weighted_tf is None:
        raise ValueError
    return weighted_tf(_tf)


def get_idf(method="log"):
    # inverse document frequency: low idf for a word appears in more docs, mean less important
    df = np.zeros((len(i2v), 1))
    for i in range(len(i2v)):
        d_count = 0
        for d in docs_words:
            d_count += 1 if i2v[i] in d else 0
        df[i, 0] = d_count

    idf_fn = idf_methods.get(method, None)
    if idf_fn is None:
        raise ValueError
    return idf_fn(df)


def cosine_similarity(q, _tf_idf):
    unit_q = q / np.sqrt(np.sum(np.square(q), axis=0, keepdims=True))
    unit_ds = _tf_idf / np.sqrt(np.sum(np.square(_tf_idf), axis=0, keepdims=True))
    similarity = unit_ds.T.dot(unit_q).ravel()
    return similarity


def docs_score(q, len_norm=False):
    q_words = q.replace(",", "").split(" ")

    # add unknown words
    unknown_v = 0
    for v in set(q_words):
        if v not in v2i:
            v2i[v] = len(v2i)
            i2v[len(v2i)-1] = v
            unknown_v += 1
    if unknown_v > 0:
        _idf = np.concatenate((idf, np.zeros((unknown_v, 1), dtype=np.float)), axis=0)
        _tf_idf = np.concatenate((tf_idf, np.zeros((unknown_v, tf_idf.shape[1]), dtype=np.float)), axis=0)
    else:
        _idf, _tf_idf = idf, tf_idf
    counter = Counter(q_words)
    q_tf = np.zeros((len(_idf), 1), dtype=np.float)     # [n_vocab, 1]
    for v in counter.keys():
        q_tf[v2i[v], 0] = counter[v]

    q_vec = q_tf * _idf            # [n_vocab, 1]

    q_scores = cosine_similarity(q_vec, _tf_idf)
    if len_norm:
        len_docs = [len(d) for d in docs_words]
        q_scores = q_scores / np.array(len_docs)
    return q_scores


def get_keywords(n=2):
    for c in range(3):
        col = tf_idf[:, c]
        idx = np.argsort(col)[-n:]
        print("doc{}, top{} keywords {}".format(c, n, [i2v[i] for i in idx]))

In [3]:
tf = get_tf()           # [n_vocab, n_doc]
idf = get_idf()         # [n_vocab, 1]
tf_idf = tf * idf       # [n_vocab, n_doc]
print("tf shape(vecb in each docs): ", tf.shape)
print("\ntf samples:\n", tf[:2])
print("\nidf shape(vecb in all docs): ", idf.shape)
print("\nidf samples:\n", idf[:2])
print("\ntf_idf shape: ", tf_idf.shape)
print("\ntf_idf sample:\n", tf_idf[:2])


# test
get_keywords()
q = "I get a coffee cup"
scores = docs_score(q)
d_ids = scores.argsort()[-3:][::-1]
print("\ntop 3 docs for '{}':\n{}".format(q, [docs[i] for i in d_ids]))

# show_tfidf(tf_idf.T, [i2v[i] for i in range(len(i2v))], "tfidf_matrix")

tf shape(vecb in each docs):  (47, 15)

tf samples:
 [[0.         0.69314718 0.69314718 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.40546511 0.         0.        ]
 [0.         0.         0.69314718 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.40546511 0.40546511 0.        ]]

idf shape(vecb in all docs):  (47, 1)

idf samples:
 [[2.32175584]
 [2.32175584]]

tf_idf shape:  (47, 15)

tf_idf sample:
 [[0.         1.60931851 1.60931851 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.94139098 0.         0.        ]
 [0.         0.         1.60931851 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.94139098 0.94139098 0.        ]]
doc0, top2 keywords ['here', 'stay']
doc1, top2 keywords ['here', 'happy']
doc2, top2 keywords ['bob', 'am']

top 3 docs for 'I get a coffee cup':
['It is coffee time, bring your cup', 'I like cof